In [11]:
from datetime import datetime

from langchain import hub
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_teddynote.models import ChatPerplexity
from langchain_teddynote.messages import stream_response
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
perplexity = ChatPerplexity(
    model="sonar",
    temperature=0.2,
    top_p=0.9,
    search_domain_filter=["perplexity.ai"],
    return_images=False,
    return_related_questions=True,
    top_k=0,
    streaming=False,
    persence_penalty=0,
    frequency_penalty=1
)

In [4]:
response = perplexity.invoke("2024년 노벨문학상 수상자를 조사해주세요")
response.content

'2024년 노벨 문학상 수상자는 대한민국의 작가 **한강**입니다. 그녀는 1970년 11월 27일 전라남도 광주에서 태어났으며, 1993년 시인으로, 1994년 소설가로 등단했습니다. 한강은 2016년 《채식주의자》로 맨부커 국제상을 수상한 바 있으며, 2024년 노벨 문학상을 수상하여 아시아인 여성으로는 최초로 이 상을 받았습니다[1][2][3]. \n\n한강의 작품은 인간의 내면과 사회적 문제를 깊이 있게 탐구하며, 독특한 문체와 상징적 인물 설정으로 주목받고 있습니다. 그녀의 수상은 한국 문학의 세계화에 큰 기여를 하였으며, 한국 문학의 국제적 인지도를 높이는 계기가 되었습니다[2][4].'

In [9]:
print(response.content)

for i, citation in enumerate(response.citations):
    print(f"[{i+1}] {citation}")

2024년 노벨 문학상 수상자는 대한민국의 작가 **한강**입니다. 그녀는 1970년 11월 27일 광주에서 태어났으며, 1993년 시인으로, 1994년 소설가로 등단했습니다. 한강은 2016년 《채식주의자》로 맨부커 국제상을 수상한 바 있으며, 2024년 노벨 문학상을 수상하여 아시아인 여성으로는 최초로, 한국인으로는 두 번째 노벨상 수상자가 되었습니다[1][2][3]. 

한강의 작품은 인간의 내면과 사회적 문제를 깊이 있게 탐구하며, 독특한 문체와 상징적 인물 설정으로 주목받고 있습니다. 그녀의 수상은 한국 문학의 세계화에 큰 기여를 하였으며, 한국 출판업계에도 긍정적인 영향을 미쳤습니다[2][4].
[1] https://ko.wikipedia.org/wiki/%ED%95%9C%EA%B0%95_(%EC%9E%91%EA%B0%80)
[2] https://www.kyeongin.com/article/1730610
[3] https://www.yna.co.kr/view/PYH20250203163500013
[4] https://www.kyeonggi.com/article/20241226580139
[5] https://www.youtube.com/watch?v=Wme6wM1tYJs


In [31]:
def pplx_print_output(response):
    print(response.content)

    for i, citation in enumerate(response.citations):
        print(f"[{i+1}] {citation}")

In [11]:
template = "{country}의 수도는 어디인가요?"

prompt = PromptTemplate.from_template(template=template)
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [12]:
prompt.format(country="대한민국")

'대한민국의 수도는 어디인가요?'

In [13]:
chain = prompt | perplexity

In [14]:
chain.invoke("대한민국").content

'대한민국의 수도는 **서울**입니다[4]. 다만, 최근에는 **세종시**를 새로운 수도로 지정해야 한다는 주장이 제기되고 있습니다. 이는 수도권 과밀화 문제 해결과 지방 발전을 위한 전략적 접근으로, 세종시가 계획도시로서의 준비성과 지리적 균형성을 갖추고 있다고 주장하고 있습니다[1][3].'

In [23]:
def get_today():
    return datetime.now().strftime("%Y년 %m월 %d일")

In [24]:
get_today()

'2025년 02월 26일'

In [35]:
prompt = PromptTemplate(
    template="{today}의 서울의 날씨를 알려주시고 날짜의 주요 뉴스를 각 {n}건씩 제공해주세요",
    input_variables=["n"],
    partial_variables={
        "today": get_today
    }
)

In [36]:
prompt.format(n=3)

'2025년 02월 26일의 서울의 날씨를 알려주시고 날짜의 주요 뉴스를 각 3건씩 제공해주세요'

In [37]:
chain = prompt | perplexity

In [40]:
response = chain.invoke(5)

In [41]:
pplx_print_output(response=response)

## 서울 날씨 (2025년 2월 26일)
서울의 날씨는 아침 최저기온이 영하 2도에서 시작하여 낮에는 영상 7도까지 오를 것으로 예상됩니다. 아침부터 맑은 날씨가 예보되며, 일교차가 크게 나타날 예정입니다[2][4].

## 주요 뉴스 (2025년 2월 26일)
현재 제공된 검색 결과에서는 2025년 2월 26일의 구체적인 뉴스가 부족합니다. 대신, 최근 관련된 몇 가지 뉴스를 소개합니다.

1. **기후정의동맹 집회**: 2025년 2월 26일, 서울시교육청 앞에서 기후정의 관련 집회가 열립니다[1].
2. **윤석열 퇴진 운동**: 최근 윤석열 퇴진과 관련된 다양한 집회와 오픈 마이크 행사가 진행되고 있습니다[1].
3. **기후위기 대응**: 기후위기와 관련된 다양한 사회적 활동과 토론이 활발히 이루어지고 있습니다[1].
4. **법률 및 정치 뉴스**: 고위공직자범죄수사처와 관련된 법적 논의가 진행 중입니다[3].
5. **날씨 및 환경**: 황사와 미세먼지 주의보가 발령되며, 기온이 점차 상승하고 있습니다[4].

추가적인 뉴스는 검색 결과에 포함되어 있지 않습니다.
[1] https://www.climatejusticealliance.kr
[2] https://www.youtube.com/watch?v=84JSSDzg6GY
[3] https://news.kbs.co.kr/news/view.do?ncd=8184114
[4] https://www.youtube.com/watch?v=mBEoAAiNtRA
[5] https://www.stroke.or.kr


### FewShotPromptTemplate

In [5]:
examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]

In [6]:
example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

print(example_prompt.format(**examples[0]))

Question:
스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인



In [7]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"]
)

question = "Google이 창립된 연도에 빌게이츠의 나이는 몇 살인가요?"
final_prompt = prompt.format(question=question)
print(final_prompt)

Question:
스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인


Question:
네이버의 창립자는 언제 태어났나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일


Question:
율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군


Question:
올드보이와 기생충의 감독이 같은 나라 출신인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예


Question:
Google이 창립된 연도에 빌게이츠의 나이는 몇 살인가요?
Answer:


In [9]:
answer = perplexity.stream(final_prompt)
stream_response(answer)

이 질문에 추가 질문이 필요한가요: 예.

추가 질문: Google은 언제 창립되었나요?
중간 답변: Google은 1998년에 창립되었습니다.

추가 질문: 빌 게이츠는 언제 태어났나요?
중간 답변: 빌 게이츠는 1955년 10월 28일에 태어났습니다.

중간 답변을 바탕으로 최종 답변을 도출합니다:

- Google이 창립된 1998년 당시, 빌 게이츠는 43세였습니다. 

최종 답변은: 43세

### Langchain Hub

In [14]:
prompt = hub.pull("rlm/rag-prompt")
print(prompt)

/workspaces/scratchs/Python/librarys/langchain/langchain-note/.venv/lib/python3.12/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [15]:
prompt = hub.pull("rlm/rag-prompt:50442af1")
prompt

/workspaces/scratchs/Python/librarys/langchain/langchain-note/.venv/lib/python3.12/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [16]:
prompt = hub.pull("teddynote/simple-summary-korean")
prompt

/workspaces/scratchs/Python/librarys/langchain/langchain-note/.venv/lib/python3.12/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'teddynote', 'lc_hub_repo': 'simple-summary-korean', 'lc_hub_commit_hash': 'b7e31df5666de7758d72fd038875973520d141548280185ee5b5ba846f015308'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='주어진 내용을 바탕으로 다음 문장을 요약하세요. 답변은 반드시 한글로 작성하세요\n\nCONTEXT: {context}\n\nSUMMARY:'), additional_kwargs={})])